# Actor-Critic

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Встает вопрос, как оценить $Q^\pi(s, a)$? В чистом policy-based алгоритме REINFORCE используется отдача $G_t$, полученная методом Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$. В Actor-Critic же предлагается отдельно обучать нейронную сеть Q-функции — критика.

Актор-критиком часто называют обобщенный фреймворк (подход), нежели какой-то конкретный алгоритм. Как подход актор-критик не указывает, каким конкретно [policy gradient] методом обучается актор и каким [value based] методом обучается критик. Таким образом актор-критик задает целое [семейство](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf) различных алгоритмов. Рекомендую в качестве шпаргалки использовать упомянутый в тетрадке с REINFORCE [пост из блога Lilian Weng](https://lilianweng.github.io/posts/2018-04-08-policy-gradient/), посвященный наиболее популярным алгоритмам семейства актор-критиков

В данной тетрадке познакомимся с наиболее простым вариантом актор-критика, который так и называют Actor-Critic:

In [1]:
# Cтавим нужные зависимости, если это колаб
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [2]:
from collections import deque

import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.distributions import Categorical

%matplotlib inline

In [3]:
env = gym.make("CartPole-v1")
env.reset()

print(f'{env.observation_space=}')
print(f'{env.action_space=}')

n_actions = env.action_space.n
state_dim = env.observation_space.shape
print(f'Action_space: {n_actions} | State_space: {env.observation_space.shape}')

env.observation_space=Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
env.action_space=Discrete(2)
Action_space: 2 | State_space: (4,)


(1 балл)

In [4]:
def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x)
    return x

def symlog(x):
    """Compute symlog values for a vector `x`. It's an inverse operation for symexp."""
    return torch.sign(x) * torch.log(torch.abs(x) + 1)

def symexp(x):
    """Compute symexp values for a vector `x`. It's an inverse operation for symlog."""
    return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)


class SymExpModule(nn.Module):
    def forward(self, x):
        return symexp(x)

def select_action_eps_greedy(Q, state, epsilon):
    """Выбирает действие epsilon-жадно."""
    if not isinstance(state, torch.Tensor):
        state = torch.tensor(state, dtype=torch.float32)
    Q_s = Q(state).detach().numpy()

    action = np.argmax(Q_s) if np.random.rand() > epsilon else np.random.choice(list(range(Q_s.shape[-1])))
    action = int(action)
    return action

def sample_batch(replay_buffer, n_samples):
    n = len(replay_buffer)
    samples_indexes = np.random.default_rng().choice(np.arange(n), size=n_samples, replace=n < n_samples)
    samples = [replay_buffer[i] for i in samples_indexes]

    states = [t[0] for t in samples]
    actions = [t[1] for t in samples]
    rewards = [t[2] for t in samples]
    next_states = [t[3] for t in samples]
    terminateds = [t[4] for t in samples]
    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(terminateds)

## Shared-body Actor-Critic

Актор и критик могут обучаться в разных режимах — актор только on-policy (шаг обучения на текущей собранной подтраектории), а критик on-policy или off-policy (шаг обучения на текущей подтраектории или на батче из replay buffer). Это с одной стороны привносит гибкость в обучение, с другой — усложняет его.

Если актор и критик оба обучаются on-policy, то имеет смысл объединить их сетки в одну и делать общий шаг обратного распространения ошибки. Однако, если они обучаются в разных режимах (и с разной частотой обновления), то велика вероятность, что их шаги обучения могут начать конфликтовать в случае общего тела — для такого варианта намного предпочтительнее разделить их на разные подсети (либо аккуратно настраивать гиперпарметры, чтобы стабилизировать обучение). В целом, рекомендуется использовать общий энкодер наблюдений, а далее как можно скорее разделять головы.

Сделаем реализацию актор-критика с общим телом и с on-policy вариантом обучения.

In [5]:
class ActorBatch:
    def __init__(self):
        self.logprobs = []
        self.q_values = []

    def append(self, log_prob, q_value):
        self.logprobs.append(log_prob)
        self.q_values.append(q_value)

    def clear(self):
        self.logprobs.clear()
        self.q_values.clear()

(3 балла)

In [6]:
def create_network(input_dim, hidden_dims):
    return nn.Sequential(
        nn.Linear(input_dim, hidden_dims[0]),
        nn.ReLU(),
        nn.Linear(hidden_dims[0], hidden_dims[1]),
        nn.ReLU()
    )

In [7]:
import torch.nn.functional as F


class ActorCriticModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()

        # Инициализируйте сеть агента с двумя головами: softmax-актора и линейного критика
        # self.net, self.actor_head, self.critic_head =
        self.net = create_network(input_dim, hidden_dims)
        self.actor_head = nn.Linear(hidden_dims[-1], output_dim)
        self.critic_head = nn.Linear(hidden_dims[-1], output_dim)
        self.eps = 5e-2
        self.eps_decay_coef = 0.95
        self.output_dim = int(output_dim)

    def eps_decay(self):
        self.eps *= self.eps_decay_coef

    def forward(self, state):
        x = self.net(state)
        logits = self.actor_head(x)
        dist = Categorical(logits=logits)
        action = dist.mode if np.random.rand() > self.eps else np.random.choice(list(range(logits.shape[-1])))
        action = int(action)
        action = torch.tensor(action)
        # action = dist.sample()
        log_prob = dist.log_prob(action)
        q_value = self.critic_head(x)[action]
        return action, log_prob, q_value

    def evaluate(self, state):
        # Вычислите значения Q-функции для данного состояния
        encoded_states = self.net(state)
        q_values = self.critic_head(encoded_states)
        return q_values

(6 баллов)

In [8]:
class ActorCriticAgent:
    def __init__(self, state_dim, action_dim, hidden_dims, lr, gamma, critic_rb_size):
        self.lr = lr
        self.gamma = gamma

        # Инициализируйте модель актор-критика и SGD оптимизатор (например, `torch.optim.Adam)`)
        self.actor_critic = ActorCriticModel(state_dim, hidden_dims, action_dim)
        self.opt = torch.optim.Adam(self.actor_critic.parameters(), lr=self.lr)

        self.actor_batch = ActorBatch()
        self.critic_rb = deque(maxlen=critic_rb_size)

    def act(self, state):
        # Произведите выбор действия и сохраните необходимые данные в батч для последующего обучения
        # Не забудьте сделать q_value.detach()
        # self.actor_batch.append(..)
        state = to_tensor(state)
        action, log_prob, q_value = self.actor_critic(state)
        self.actor_batch.append(log_prob, q_value.detach())
        return action.item()

    def append_to_replay_buffer(self, s, a, r, next_s, terminated):
        # Добавьте новый экземпляр данных в память прецедентов.
        self.critic_rb.append((s, a, r, next_s, terminated))

    def evaluate(self, state):
        return self.actor_critic.evaluate(state)

    def update(self, rollout_size, critic_batch_size, critic_updates_per_actor):
        if len(self.actor_batch.q_values) < rollout_size:
            return None, None

        actor_loss  = self.update_actor()
        critic_loss = self.update_critic(critic_batch_size, critic_updates_per_actor)

        # self.opt.zero_grad()
        # critic_loss = self.update_critic(critic_batch_size, critic_updates_per_actor)
        # actor_loss  = self.update_actor()
        # loss = critic_loss + actor_loss
        # loss.backward()

        # self.opt.step()
        # self.actor_batch.clear()
        # self.critic_rb.clear()

        return critic_loss.detach(), actor_loss.detach()

    def update_actor(self):
        Q_s_a = to_tensor(self.actor_batch.q_values)
        logprobs = torch.stack(self.actor_batch.logprobs)

        # Реализуйте шаг обновления актора — вычислите ошибку `loss` и произведите шаг обновления градиентным спуском.
        advantages = Q_s_a - Q_s_a.mean()
        loss = -(logprobs * advantages).mean()

        self.opt.zero_grad()
        loss.backward()
        self.opt.step()
        self.actor_batch.clear()

        return loss

    def update_critic(self, batch_size, n_updates=1):
        # Реализуйте n_updates шагов обучения критика.
        loss = torch.tensor(0.0)
        for _ in range(n_updates):
            batch = sample_batch(self.critic_rb, batch_size)
            td_loss = self.compute_td_loss(*batch)

            self.opt.zero_grad()
            td_loss.backward()
            self.opt.step()
            loss += td_loss
        return loss / n_updates
        
        # loss = torch.tensor(0.0)
        # for _ in range(n_updates):
        #     batch = sample_batch(self.critic_rb, batch_size)
        #     td_loss = self.compute_td_loss(*batch)
        #     loss += td_loss
        # return loss / n_updates

    def compute_td_loss(
        self, states, actions, rewards, next_states, terminateds, regularizer=0.1
    ):
        # переводим входные данные в тензоры
        s = to_tensor(states)                     # shape: [batch_size, state_size]
        a = to_tensor(actions, int).long()        # shape: [batch_size]
        r = to_tensor(rewards)                    # shape: [batch_size]
        s_next = to_tensor(next_states)           # shape: [batch_size, state_size]
        term = to_tensor(terminateds, bool)       # shape: [batch_size]

        mask = (~term).float()
        # получаем Q[s, a] для выбранных действий в текущих состояниях (для каждого примера из батча)
        # Q_s_a = ...
        Q_s_a = self.evaluate(s)[:, a]

        # with torch.no_grad():
        # получаем Q[s_next, *] — значения полезности всех действий в следующих состояниях
        Q_sn = self.evaluate(s_next)

        # а затем вычисляем V*[s_next] — оптимальные значения полезности следующих состояний
        V_sn = torch.max(Q_sn, dim=-1)[0]

        # вычисляем TD target и далее TD error
        target = r + self.gamma * V_sn * mask
        td_error = target - Q_s_a

        # MSE loss для минимизации
        loss = torch.mean(td_error ** 2)

        # добавляем регуляризацию на значения Q
        loss += regularizer * Q_s_a.mean()
        return loss

In [9]:
def run_actor_critic(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=5e-4,
        total_max_steps=200_000,
        train_schedule=16, replay_buffer_size=50000, batch_size=64, critic_updates_per_actor=4,
        eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    episode_return_history = deque(maxlen=smooth_ret_window)

    agent = ActorCriticAgent(
        state_dim=env.observation_space.shape[0], action_dim=env.action_space.n, hidden_dims=hidden_dims,
        lr=lr, gamma=.995, critic_rb_size=replay_buffer_size
    )

    s, _ = env.reset()
    done, episode_return = False, 0.
    eval = False

    last_critic_loss = None
    last_actor_loss = None
    for global_step in range(1, total_max_steps+1):
        a = agent.act(s)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode_return += r
        done = terminated or truncated

        # train step
        agent.append_to_replay_buffer(s, a, r, s_next, terminated)
        critic_loss, actor_loss = agent.update(train_schedule, batch_size, critic_updates_per_actor)
        if critic_loss is not None:
            last_critic_loss = critic_loss
            last_actor_loss = actor_loss

        # evaluate
        if global_step % eval_schedule == 0:
            agent.actor_critic.eps_decay()
            eval = True

        s = s_next
        if done:
            if eval:
                episode_return_history.append(episode_return)
                avg_return = np.mean(episode_return_history)
                eps = agent.actor_critic.eps
                print(f'{global_step=} | {avg_return=:.3f} | {last_critic_loss=:.3f} | {last_actor_loss=:.3f} | {eps=:.5f}')
                if avg_return >= success_ret:
                    print('Решено!')
                    break

            s, _ = env.reset()
            done, episode_return = False, 0.
            eval = False

run_actor_critic(train_schedule=16, eval_schedule=2000, total_max_steps=200_000)

global_step=2001 | avg_return=9.000 | last_critic_loss=0.918 | last_actor_loss=-0.018 | eps=0.04750
global_step=4011 | avg_return=11.500 | last_critic_loss=0.962 | last_actor_loss=-0.057 | eps=0.04512
global_step=6003 | avg_return=13.333 | last_critic_loss=0.936 | last_actor_loss=-0.029 | eps=0.04287
global_step=8009 | avg_return=13.000 | last_critic_loss=0.926 | last_actor_loss=-0.054 | eps=0.04073
global_step=10004 | avg_return=13.600 | last_critic_loss=0.931 | last_actor_loss=-0.028 | eps=0.03869
global_step=12002 | avg_return=13.000 | last_critic_loss=0.939 | last_actor_loss=-0.019 | eps=0.03675
global_step=14000 | avg_return=15.143 | last_critic_loss=0.919 | last_actor_loss=-0.006 | eps=0.03492
global_step=16002 | avg_return=15.000 | last_critic_loss=0.940 | last_actor_loss=-0.029 | eps=0.03317
global_step=18011 | avg_return=15.111 | last_critic_loss=0.961 | last_actor_loss=0.092 | eps=0.03151
global_step=20001 | avg_return=14.700 | last_critic_loss=0.917 | last_actor_loss=-0.032 